In [1]:
import ctypes

In [2]:
mem_loc = id(4)

mem = []
for x in range(256):
    mem.append(ctypes.c_char.from_address(mem_loc + x).value[0])
mem = bytes(mem)

In [3]:
spacing = 32  # emperical
for n in range(len(mem) // spacing):
    print(mem[n*spacing:(n+1)*spacing].hex())

e2010000000000008077806e4f56000001000000000000000400000000000000
15010000000000008077806e4f56000001000000000000000500000000000000
f0000000000000008077806e4f56000001000000000000000600000000000000
ae000000000000008077806e4f56000001000000000000000700000000000000
fc000000000000008077806e4f56000001000000000000000800000000000000
89000000000000008077806e4f56000001000000000000000900000000000000
94000000000000008077806e4f56000001000000000000000a00000000000000
85000000000000008077806e4f56000001000000000000000b00000000000000


In [4]:
mem[24]

4

In [5]:
mem[spacing + 24]

5

Hmmmm